In [1]:
import pandas as pd
import jieba
import re
import matplotlib
import matplotlib.pyplot as plt
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pyLDAvis.gensim_models
import numpy as np

from pymongo import MongoClient
import pymongo

from gensim.test.utils import datapath

In [2]:
# 使用GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
! nvidia-smi

Thu Jul  7 16:20:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 64%   70C    P2    82W / 260W |   2071MiB / 11019MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 29%   

# MongoDB

In [115]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_rm_de = db.rm_patent_de
    lda_topic_de = db.lda_topic_de


except errors.ConnectionFailure as err:
    print(err)

In [138]:
df_lda_topic_de = pd.DataFrame(list(lda_topic_de.find()))

In [139]:
df_lda_topic_de

,_id,patentNumber,appDate,company,classification,brief,topic
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]","[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,6
1,62c6c499afc6a3b84b1e6de0,DE102012022880A1,2012-11-23 05:00:00,[Merck Patent GmbH],"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,10
2,62c6c499afc6a3b84b1e6de1,DE102012218966B4,2012-10-18 04:00:00,[International Business Machines Corp],"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,4
3,62c6c499afc6a3b84b1e6de2,DE112012005533B4,2012-10-31 04:00:00,[International Business Machines Corp],"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,11
4,62c6c499afc6a3b84b1e6de3,DE102012021919B4,2012-11-09 05:00:00,"[M2C Solutions GmbH, ITK Engineering GmbH]","[G07C5/008, G06Q10/00]",Die Erfindung betrifft eine Scoring-Vorrichtun...,1
...,...,...,...,...,...,...,...
1060,62c6c499afc6a3b84b1e7203,DE102020126369A1,2020-10-08 04:00:00,[Toyota Motor Corp],"[B60L53/64, B60L53/60, B60L50/60, B60L53/12, B...",Ein Server (2) weist eine Ladeablaufplanerzeug...,9
1061,62c6c499afc6a3b84b1e7204,DE102020112735A1,2020-05-11 04:00:00,[Honeywell International Inc],"[A61M5/16854, A61M39/08, A61M5/365, G01N29/02,...",Ein Fluidmessgerät und ein Verfahren zum Erfas...,5
1062,62c6c499afc6a3b84b1e7205,DE202020002356U1,2020-05-28 04:00:00,[],[G06F9/45529],None,6
1063,62c6c499afc6a3b84b1e7206,DE102020104109A1,2020-02-17 05:00:00,[Geiger Bauwerksanierung & Co Kg GmbH],"[C23F13/20, C23F13/04, E04C5/015, G01N33/383, ...",Die vorliegende Erfindung betrifft ein System ...,5


In [140]:
df_de_all = df_lda_topic_de.explode('classification')

In [141]:
df_de_all.head(3)

,_id,patentNumber,appDate,company,classification,brief,topic
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/3234,None,6
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/329,None,6
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0216,None,6


In [142]:
sum(df_de_all.classification.isna())

15

In [143]:
len(df_de_all)

8099

In [150]:
df_de_all_n = df_de_all[df_de_all["classification"].notna()].copy()
df_de_all_n

,_id,patentNumber,appDate,company,classification,brief,topic
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/3234,None,6
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/329,None,6
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0216,None,6
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04L43/08,None,6
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0219,None,6
...,...,...,...,...,...,...,...
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/09,Es wird eine piezoelektrische Zusammensetzung ...,3
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/113,Es wird eine piezoelektrische Zusammensetzung ...,3
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/187,Es wird eine piezoelektrische Zusammensetzung ...,3
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H03H9/17,Es wird eine piezoelektrische Zusammensetzung ...,3


In [196]:
# 取cpc前3碼
df_de_all_n["cpc_n"] = df_de_all_n.classification.apply(lambda x: x[0:11])

In [197]:
df_de_all_n

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/3234,None,6,G06F1/3234
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/329,None,6,G06F1/329
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0216,None,6,H04W52/0216
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04L43/08,None,6,H04L43/08
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0219,None,6,H04W52/0219
...,...,...,...,...,...,...,...,...
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/09,Es wird eine piezoelektrische Zusammensetzung ...,3,H01L41/09
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/113,Es wird eine piezoelektrische Zusammensetzung ...,3,H01L41/113
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/187,Es wird eine piezoelektrische Zusammensetzung ...,3,H01L41/187
1064,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H03H9/17,Es wird eine piezoelektrische Zusammensetzung ...,3,H03H9/17


In [210]:
df_de_all_n = df_de_all_n.sort_values("topic")

In [211]:
df_de_all_n

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
332,62c6c499afc6a3b84b1e6f2b,DE202017006244U1,2017-12-05 05:00:00,[SESAM GmbH],G07F17/13,Paketbox (10) für und mit mindestens einem aus...,0,G07F17/13
334,62c6c499afc6a3b84b1e6f2d,DE102017126457A1,2017-11-10 05:00:00,[Ars Computer und Consulting GmbH],G16H40/20,Es wird eine Vorrichtung (1) zur Unterstützung...,0,G16H40/20
334,62c6c499afc6a3b84b1e6f2d,DE102017126457A1,2017-11-10 05:00:00,[Ars Computer und Consulting GmbH],Y02P90/30,Es wird eine Vorrichtung (1) zur Unterstützung...,0,Y02P90/30
775,62c6c499afc6a3b84b1e70e6,DE112019002417T5,2019-06-25 04:00:00,[Intel Corp],G11C7/222,"Es ist eine Vorrichtung vorgesehen, wobei die ...",0,G11C7/222
525,62c6c499afc6a3b84b1e6fec,DE102018114660B3,2018-06-19 04:00:00,[Techem Energy Services GmbH],G01R31/3832,Es wird eine Datensammelvorrichtung (1) zum Er...,0,G01R31/3832
...,...,...,...,...,...,...,...,...
763,62c6c499afc6a3b84b1e70da,DE112019000104T5,2019-10-03 04:00:00,[Sony Group Corp],G11B5/7022,"Eine Aufgabe besteht darin, ein magnetisches A...",12,G11B5/7022
763,62c6c499afc6a3b84b1e70da,DE112019000104T5,2019-10-03 04:00:00,[Sony Group Corp],G11B5/70678,"Eine Aufgabe besteht darin, ein magnetisches A...",12,G11B5/70678
763,62c6c499afc6a3b84b1e70da,DE112019000104T5,2019-10-03 04:00:00,[Sony Group Corp],G11B5/714,"Eine Aufgabe besteht darin, ein magnetisches A...",12,G11B5/714
764,62c6c499afc6a3b84b1e70db,DE112019003234T5,2019-05-27 04:00:00,[Sony Corp],G06F3/013,Diese Audioverarbeitungsvorrichtung umfasst: e...,12,G06F3/013


In [212]:
df_de_all_n.to_csv("lda_all_result/find_topic_cpc_de.csv",index=0)

In [213]:
df_topic = pd.read_csv('lda_all_result/find_topic_cpc_de.csv')

In [214]:
df_topic.tail()

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
8079,62c6c499afc6a3b84b1e70da,DE112019000104T5,2019-10-03 04:00:00,['Sony Group Corp'],G11B5/7022,"Eine Aufgabe besteht darin, ein magnetisches A...",12,G11B5/7022
8080,62c6c499afc6a3b84b1e70da,DE112019000104T5,2019-10-03 04:00:00,['Sony Group Corp'],G11B5/70678,"Eine Aufgabe besteht darin, ein magnetisches A...",12,G11B5/70678
8081,62c6c499afc6a3b84b1e70da,DE112019000104T5,2019-10-03 04:00:00,['Sony Group Corp'],G11B5/714,"Eine Aufgabe besteht darin, ein magnetisches A...",12,G11B5/714
8082,62c6c499afc6a3b84b1e70db,DE112019003234T5,2019-05-27 04:00:00,['Sony Corp'],G06F3/013,Diese Audioverarbeitungsvorrichtung umfasst: e...,12,G06F3/013
8083,62c6c499afc6a3b84b1e6fcf,DE102018009365A1,2018-11-29 05:00:00,['Giesecke and Devrient Mobile Security GmbH'],G06F21/575,Die vorliegende Erfindung ist gerichtet auf ei...,12,G06F21/575


## 找出topic最多的CPC

In [198]:
df_de_all_n.groupby(["topic","cpc_n"],as_index=False).size()

,topic,cpc_n,size
0,0,A47G2029/14,4
1,0,A47G29/141,2
2,0,B65D81/18,1
3,0,F25D23/069,1
4,0,F25D2303/08,1
...,...,...,...
6587,12,Y02D30/70,2
6588,12,Y02E60/10,1
6589,12,Y02P90/02,2
6590,12,Y04S20/12,1


In [199]:
df_de_all_s = df_de_all_n.groupby(["topic","cpc_n"],as_index=False).size()
df_de_all_s

,topic,cpc_n,size
0,0,A47G2029/14,4
1,0,A47G29/141,2
2,0,B65D81/18,1
3,0,F25D23/069,1
4,0,F25D2303/08,1
...,...,...,...
6587,12,Y02D30/70,2
6588,12,Y02E60/10,1
6589,12,Y02P90/02,2
6590,12,Y04S20/12,1


In [209]:
df_de_all_s[df_de_all_s["topic"]==11].sort_values("size",ascending=False)

,topic,cpc_n,size
6106,11,H04W4/70,5
5857,11,G06T2207/20,4
5927,11,H01L2224/48,4
5724,11,F01N3/2066,3
6054,11,H04L67/12,3
...,...,...,...
5829,11,G06K9/0002,1
5828,11,G06K9/00013,1
5827,11,G06K7/1417,1
5826,11,G06K19/0615,1


In [201]:
cpc_max = df_de_all_s.groupby(["topic"],as_index=False)["size"].agg('max')
cpc_max

,topic,size
0,0,4
1,1,10
2,2,10
3,3,3
4,4,3
5,5,6
6,6,4
7,7,14
8,8,8
9,9,3


In [202]:
df_de_all_s.merge(cpc_max,on = ["topic","size"])

,topic,cpc_n,size
0,0,A47G2029/14,4
1,1,G05B2219/25,10
2,1,H04L67/12,10
3,2,H01L2224/29,10
4,3,B60K2370/18,3
5,3,B60W2040/08,3
6,3,B60W2050/00,3
7,3,G05B2219/31,3
8,4,H04L5/0055,3
9,4,H04L67/12,3


In [203]:
# 轉成 Year-topic 的總數表格
table_de = df_de_all_s.pivot_table(index='topic', values='size', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'topic'})

In [204]:
table_de

cpc_n,topic,A01G25/167,A01G27/003,A01G27/006,A01G27/02,A01G31/06,A01G7/045,A01G9/02,A01G9/022,A01G9/025,...,Y04S20/30,Y04S30/12,Y04S30/14,Y04S40/18,Y04S40/20,Y10S160/06,Y10T29/4911,Y10T428/291,Y10T442/30,Y10T442/397
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,3.0,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [206]:
table_de.to_csv('lda_all_result/lda_brief_de_all_topic_cpc_11.csv',index=0)